In [12]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import json
import time
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
import pandas as pd

In [13]:
options = webdriver.ChromeOptions()
options.add_argument('headless')#창 안 뜨게 설정
chromedriver_path = "chromedriver"
review_driver = webdriver.Chrome('C:/Users/daye/practice/chromedriver.exe',options=options)# 크롬 경로

In [29]:
#홈에서의 더보기 경로는 홈페이지마다 위치가 달라지므로 f12 버튼 눌러서 파악해야 함
def scroll_click(review_driver, see_more_xpath):
    #홈에서 스크롤 다운
    review_driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    #홈에서 방문자 리뷰 더보기 클릭
    review_driver.find_element_by_xpath(see_more_xpath).click()
    #더보기 탭 끝까지 클릭하기 
    while True:
        try:
            review_driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[5]/div[4]/div[5]/div[2]/a').click()
        except:
            break

In [38]:
def crawling(store_number, see_more_xpath):
    #크롬드라이버 구동하기 
    review_driver.implicitly_wait(imp_time)
    
    #url 접속
    review_url = 'https://pcmap.place.naver.com/restaurant/'+store_number+'/home?from=map&amp;fromPanelNum=2&amp'
    review_driver.get(review_url)
    time.sleep(5)
    
    """특징 별 리뷰 뽑아오기"""
    
    """맛"""
    #맛 클릭하기
    taste_button = review_driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[5]/div[4]/div[5]/div[1]/div[1]/div/a[2]')
    taste_button.click()
    
    html = review_driver.page_source
    # print(html)
    
    #모듈 참조
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html, 'lxml')
    
    #댓글추출
    tastes = review_driver.find_elements_by_class_name('highlight')
    tastes = [taste.text for taste in tastes]
    
    #작성자
    nicks = review_driver.find_elements_by_class_name('hbo4A')
    nicks = [nick.text for nick in nicks]
    
    #취합
    taste_review = list(zip(nicks, tastes))
    
    driver.quit()
    return taste_review

    """서비스"""
    #서비스 클릭하기
    driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[5]/div[4]/div[5]/div[1]/div[1]/div/a[3]').send_keys(Keys.ENTER)
    
       
    #댓글추출
    services = driver.find_elements_by_class_name('highlight')
    services = [services.text for service in services]

    #취합
    service_review = list(zip(nicks, services))

    """ 만족도 """
    #만족도 클릭하기
    driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[5]/div[4]/div[5]/div[1]/div[1]/div/a[11]').send_keys(Keys.ENTER)
       
    #댓글추출
    satisfactions = driver.find_elements_by_class_name('highlight')
    satisfactions = [services.text for service in services]
    
    #취합
    satisfaction_review = list(zip(nicks,satisfactions))
    driver.quit()
    
    #데이터 프레임으로 만들기
    taste_review = pd.DataFrame({'author': nicks, 'tastes': tastes, 'services': services, 'satisfaction': satisfaction})
    
    return taste_review